In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests, re
import itertools as it
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support import expected_conditions as EC
import warnings
warnings.filterwarnings('ignore')
import random
import time
import math


In [26]:
df = pd.read_csv('LinkedIn_excel_processed.csv')
df_missing = df[(df['# employees']=='na')|(df['# employees'].isnull())]
df_missing

,name,LinkedIn,industry,job_openings,headcount,hlfyr,oneyr,twoyr,# employees
26,Stitch Labs,http://www.linkedin.com/company/stitch-labs-inc-,e-commerce,na,0,na,na,na,na
44,Full Harvest,https://www.linkedin.com/company/full-harvest-...,e-commerce,na,0,na,na,na,na
72,Maisonette,https://www.linkedin.com/in/luisana-mendoza-de...,e-commerce,na,0,na,na,na,na
98,Boomerang Commerce,http://www.linkedin.com/company/boomerangcommerce,e-commerce,na,0,na,na,na,na
122,The Catch Co,https://in.linkedin.com/company/mystery-tackle...,e-commerce,na,0,na,na,na,na
...,...,...,...,...,...,...,...,...,...
808,KlickThru,http://www.linkedin.com/company/klickthru,social media,NaN,0,NaN,NaN,NaN,NaN
816,Skytide,https://www.linkedin.com/company/skytide-inc,social media,NaN,0,NaN,NaN,NaN,NaN
818,White Label Marketing,https://www.linkedin.com/company/white-label-m...,social media,NaN,0,NaN,NaN,NaN,NaN
819,Gizmorati,http://www.linkedin.com/company/gizmorati,social media,NaN,0,NaN,NaN,NaN,NaN


In [11]:
Search_link = ['https://www.linkedin.com/search/results/companies/?keywords='+i for i in df_missing['name']]

In [16]:
driver = webdriver.Chrome()
driver.get(Search_link[1])

In [46]:
emps = []
jobs = []
hlf = []
one = []
two = []
cnt = 1
for i in Search_link:
    try:
        driver.get(i)
        element_click = EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div[3]/div/div[2]/div/div/div/div/div[2]/ul/li[1]/div/div/div/div[2]/div[1]/div/div[1]/span/div/span/span/a'))
        WebDriverWait(driver, 5).until(element_click)
        first_result = driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[2]/div/div/div/div/div[2]/ul/li[1]/div/div/div/div[2]/div[1]/div/div[1]/span/div/span/span/a')
        first_result.click()

        element_click = EC.element_to_be_clickable((By.LINK_TEXT, 'Insights'))
        WebDriverWait(driver, 5).until(element_click)
        menu = driver.find_element_by_class_name('org-grid__left-rail')

        try:
            employees = driver.find_element_by_partial_link_text('See all').text
        except:
            employees = ''

        menu.find_element_by_link_text('Jobs').click()

        try:
            element_present = EC.presence_of_element_located((By.CLASS_NAME, 'org-jobs-job-search-form-module__headline'))
            WebDriverWait(driver, 5).until(element_present)
            job_openings = int(driver.find_element_by_class_name('org-jobs-job-search-form-module__headline').text.split('job')[0].strip().split(' ')[-1])
        except:
            job_openings =''

        menu.find_element_by_link_text('Insights').click()

        try:
            element_present = EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'td.org-insights-module__summary-block'))
            WebDriverWait(driver, 5).until(element_present)
            yr_sum = driver.find_elements_by_css_selector('td.org-insights-module__summary-block')
            try:
                hal=yr_sum[1].text
            except:
                hal=''
            try:  
                on=yr_sum[2].text
            except:
                on=''
            try:
                tw=yr_sum[3].text
            except:
                tw=''
            print('No.'+str(cnt)+' Finished')
        except: 
            hal=''
            on=''
            tw=''
            print('No.'+str(cnt)+' Insights Page Missing Info')
    except:
        employees=''
        job_openings=''
        hal=''
        on=''
        tw=''
        print('No.'+str(cnt)+' Invalid Search')

    emps.append(employees)
    jobs.append(job_openings)
    hlf.append(hal)
    one.append(on)
    two.append(tw)

    cnt+=1

No.1 Insights Page Missing Info
No.2 Finished
No.3 Finished
No.4 Finished
No.5 Insights Page Missing Info
No.6 Finished
No.7 Insights Page Missing Info
No.8 Finished
No.9 Finished
No.10 Finished
No.11 Finished
No.12 Insights Page Missing Info
No.13 Insights Page Missing Info
No.14 Invalid Search
No.15 Insights Page Missing Info
No.16 Insights Page Missing Info
No.17 Insights Page Missing Info
No.18 Insights Page Missing Info
No.19 Insights Page Missing Info
No.20 Insights Page Missing Info
No.21 Invalid Search
No.22 Insights Page Missing Info
No.23 Insights Page Missing Info
No.24 Insights Page Missing Info
No.25 Invalid Search
No.26 Invalid Search
No.27 Invalid Search
No.28 Insights Page Missing Info
No.29 Insights Page Missing Info
No.30 Invalid Search
No.31 Insights Page Missing Info
No.32 Insights Page Missing Info
No.33 Invalid Search
No.34 Insights Page Missing Info
No.35 Invalid Search
No.36 Finished
No.37 Finished
No.38 Finished
No.39 Finished
No.40 Finished
No.41 Invalid Searc

In [47]:
df_missing['# employees'] = emps
df_missing['job_openings'] = jobs
df_missing['hlfyr'] = hlf
df_missing['oneyr'] = one
df_missing ['twoyr'] = two

In [50]:
for i in df_missing.index.to_list():
    df['# employees'][i]  = df_missing['# employees'][i]
    df['job_openings'][i] = df_missing['job_openings'][i]
    df['hlfyr'][i]  = df_missing['hlfyr'][i]
    df['oneyr'][i]  = df_missing['oneyr'][i]
    df['twoyr'][i]  = df_missing['twoyr'][i]


,name,LinkedIn,industry,job_openings,headcount,hlfyr,oneyr,twoyr,# employees
0,Finix,https://www.linkedin.com/company/finix-payments/,e-commerce,7,90,17%\n17% increase,105%\n105% increase,429%\n429% increase,See all 90 employees on LinkedIn
1,Scentbird,http://www.linkedin.com/company/4997584,e-commerce,NaN,89,1%\n1% increase,3%\n-3% decrease,10%\n10% increase,See all 89 employees on LinkedIn
2,Beauty Bakerie Cosmetics Brand,https://www.linkedin.com/company/beauty-bakeri...,e-commerce,NaN,26,NaN,NaN,NaN,See all 26 employees on LinkedIn
3,Goldbelly,https://www.linkedin.com/company/goldbelly/,e-commerce,15,49,20%\n20% increase,29%\n29% increase,58%\n58% increase,See all 49 employees on LinkedIn
4,Tophatter,https://www.linkedin.com/company/tophatter-com,e-commerce,3,107,5%\n5% increase,9%\n9% increase,2%\n-2% decrease,See all 107 employees on LinkedIn
...,...,...,...,...,...,...,...,...,...
816,Skytide,https://www.linkedin.com/company/skytide-inc,social media,,0,,,,
817,Accordent Technologies,https://www.linkedin.com/company/accordent-tec...,social media,NaN,11,NaN,NaN,NaN,See all 11 employees on LinkedIn
818,White Label Marketing,https://www.linkedin.com/company/white-label-m...,social media,,0,,,,See all 15 employees on LinkedIn
819,Gizmorati,http://www.linkedin.com/company/gizmorati,social media,,0,,,,See all


In [54]:
df.to_csv('LinkedIn_3.csv',index=False)